### Check which method better suits NN2 model, Oversampling or Undersampling

In [ ]:
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
X_train, X_validation, y_train, y_validation = train_test_split(X_tv, y_tv,test_size=0.25, random_state=random_state)

cv = KFold(n_splits=5,random_state=random_state,shuffle=True)
collection = []
important_dict = {}

# NN2 (Baseline Model)

model_nn2 = Sequential()
model_nn2.add(Dense(42, activation='tanh', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))) # Input layer
model_nn2.add(Dropout(0.2))
model_nn2.add(Dense(5, activation='tanh', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))) # hidden layer 1
model_nn2.add(Dropout(0.2))
model_nn2.add(Dense(5, activation='tanh', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))) # hidden layer 2
model_nn2.add(Dropout(0.2))
model_nn2.add(Dense(1, activation='sigmoid')) # Output layer
model_nn2.compile(loss=BinaryCrossentropy(from_logits=False), optimizer='adam', metrics=[f1, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

features = list(X_train.columns)

X_train = np.array(X_train)
y_train = np.array(y_train)

for method in ['Oversample','Undersample','Unchanged']:
    one_result = {}
    important_dict['method'] = method
    if method == 'Unchanged':
        one_result['method'] = method
        print(method)
        model_ap_list = []
        model_f1_list = []

        for fold_train_index,fold_test_index in cv.split(X_train):

            X_train_fold = X_train[fold_train_index]
            X_test_fold = X_train[fold_test_index]
            y_train_fold = y_train[fold_train_index]
            y_test_fold = y_train[fold_test_index]
            model_nn2.fit(X_train_fold, y_train_fold, epochs=5, callbacks=[EarlyStopping(patience=2)])

            fold_pred_prob = model_nn2.predict(X_test_fold)
            class_pred = np.where(fold_pred_prob > 0.5, 1, 0)[:, 0]
            model_f1 = f1_score(y_test_fold, class_pred)
            model_ap = model_nn2.evaluate(X_test_fold, y_test_fold, verbose=0)[2]

            print('Average Precision:',model_ap)
            print('F1 Score:', model_f1)
            model_ap_list.append(model_ap)
            model_f1_list.append(model_f1)
        one_result['ap'] = np.mean(model_ap)
        one_result['f1'] = np.mean(model_f1)
        collection.append(one_result)


    if method == 'Oversample':
        one_result['method'] = method
        print(method)
        model_ap_list = []
        oversampler = RandomOverSampler(sampling_strategy='minority')
        model_f1_list = []

        for fold_train_index,fold_test_index in cv.split(X_train):

            X_train_fold = X_train[fold_train_index]
            X_test_fold = X_train[fold_test_index]
            y_train_fold = y_train[fold_train_index]
            y_test_fold = y_train[fold_test_index]

            X_train_fold,y_train_fold = oversampler.fit_resample(X_train_fold,y_train_fold)
            model_nn2.fit(X_train_fold,y_train_fold, epochs=5, callbacks=[EarlyStopping(patience=2)])

            fold_pred_prob = model_nn2.predict(X_test_fold)
            class_pred = np.where(fold_pred_prob > 0.5, 1, 0)[:, 0]
            model_f1 = f1_score(y_test_fold, class_pred)
            model_ap = model_nn2.evaluate(X_test_fold, y_test_fold, verbose=0)[2]

            print('Average Precision:',model_ap)
            print('F1 Score:', model_f1)
            model_ap_list.append(model_ap)
            model_f1_list.append(model_f1)
        one_result['ap'] = np.mean(model_ap)
        one_result['f1'] = np.mean(model_f1)
        collection.append(one_result)

    if method == 'Undersample':
        one_result['method'] = method
        print(method)
        model_ap_list = []
        undersampler = RandomUnderSampler(sampling_strategy='majority')
        model_f1_list = []

        for fold_train_index,fold_test_index in cv.split(X_train):

            X_train_fold = X_train[fold_train_index]
            X_test_fold = X_train[fold_test_index]
            y_train_fold = y_train[fold_train_index]
            y_test_fold = y_train[fold_test_index]

            X_train_fold,y_train_fold = undersampler.fit_resample(X_train_fold,y_train_fold)
            model_nn2.fit(X_train_fold,y_train_fold, epochs=5, callbacks=[EarlyStopping(patience=2)])

            fold_pred_prob = model_nn2.predict(X_test_fold)
            class_pred = np.where(fold_pred_prob > 0.5, 1, 0)[:, 0]
            model_f1 = f1_score(y_test_fold, class_pred)
            model_ap = model_nn2.evaluate(X_test_fold, y_test_fold, verbose=0)[2]

            print('Average Precision:',model_ap)
            print('F1 Score:', model_f1)
            model_ap_list.append(model_ap)
            model_f1_list.append(model_f1)
        one_result['ap'] = np.mean(model_ap)
        one_result['f1'] = np.mean(model_f1)
        collection.append(one_result)

### Based on F1 Score, we know that Oversampling better suits NN2 model

In [73]:
rebalance_result = pd.DataFrame(collection)
rebalance_result

,method,ap,f1
0,Oversample,0.425917,0.522192
1,Undersample,0.419024,0.520993
2,Unchanged,0.618102,0.187910
